In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import json

ENDPOINT = "https://coco-subset-training.cognitiveservices.azure.com/"

# Replace with a valid key
training_key = "MY_TRAINING_KEY"
prediction_key = "MY_TESTING_KEY"
prediction_resource_id = "MY_RESOURCE_ID"

publish_iteration_name = "detectModel"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
project = trainer.create_project("My Detection Project", domain_id=obj_detection_domain.id)

Creating project...


In [ ]:
tags = ['sports ball', 'frisbee', 'bowl', 'orange', 'apple', 'donut']
ball_tag = trainer.create_tag(project.id, "sports ball")
frisbee_tag = trainer.create_tag(project.id, "frisbee")
bowl_tag = trainer.create_tag(project.id, "bowl")
orange_tag = trainer.create_tag(project.id, "orange")
apple_tag = trainer.create_tag(project.id, "apple")
donut_tag = trainer.create_tag(project.id, "donut")

tag_dict = {tags[i]:tag for i, tag in enumerate([ball_tag, frisbee_tag, bowl_tag, orange_tag, apple_tag, donut_tag])}
tag_dict

In [ ]:
# need to fetch images by tag and then upload
def GetImageNameMapping():
    path = "D:/annotations_subset/instances_train2014.json"
    with open(path, 'r') as fp:
        data = json.load(fp)
    data.keys()
    from pandas import json_normalize
    category_ids = json_normalize(data['categories'])
    category_ids = category_ids[['id', 'name']]
    category_ids = {row['id']: row['name'] for index, row in category_ids.iterrows()}
    
    path = "D:/annotations/instances_train2014.json"
    with open(path, 'r') as fp:
        data = json.load(fp)
    data.keys()
    image_ids = {x['image_id'] : x['category_id'] for x in data['annotations'] if x['category_id'] in category_ids.keys()}
    bboxes = {x['image_id'] : x['bbox'] for x in data['annotations'] if x['category_id'] in category_ids.keys()}
    files = []
    for x in data['images']:
        if x['id'] in image_ids.keys():
            
            files.append({'file_name':x['file_name'], 'tag' : category_ids[image_ids[x['id']]],'bbox': bboxes[x['id']]})
            
    return json_normalize(files)

In [48]:
path = "D:/annotations_subset/instances_train2014.json"
with open(path, 'r') as fp:
    data = json.load(fp)
data.keys()
from pandas import json_normalize
category_ids = json_normalize(data['categories'])
category_ids = category_ids[['id', 'name']]
category_ids = {row['id']: row['name'] for index, row in category_ids.iterrows()}

path = "D:/annotations/instances_train2014.json"
with open(path, 'r') as fp:
    data = json.load(fp)
data.keys()
image_ids = {x['image_id'] : category_ids[x['category_id']] for x in data['annotations'] if x['category_id'] in category_ids.keys()}
bboxes = {x['image_id'] : x['bbox'] for x in data['annotations'] if x['category_id'] in category_ids.keys()}
bboxes = [(x['file_name'],image_ids[x['id']], bboxes[x['id']]) for x in data['images'] if x['id'] in image_ids.keys()]

KeyboardInterrupt: 

{'sports ball': <azure.cognitiveservices.vision.customvision.training.models._models_py3.Tag at 0x25894d5d348>,
 'frisbee': <azure.cognitiveservices.vision.customvision.training.models._models_py3.Tag at 0x25894d5c908>,
 'bowl': <azure.cognitiveservices.vision.customvision.training.models._models_py3.Tag at 0x25894d58888>,
 'orange': <azure.cognitiveservices.vision.customvision.training.models._models_py3.Tag at 0x25894d5f808>,
 'apple': <azure.cognitiveservices.vision.customvision.training.models._models_py3.Tag at 0x25894d60788>,
 'donut': <azure.cognitiveservices.vision.customvision.training.models._models_py3.Tag at 0x25894d63708>}

In [25]:
tagged_images_with_regions = []
for items in bboxes:
    file_name = items[0]
    filepath = f'D:/train2014_subset/{file_name}'
    tag = items[1]
    x, y, w, h = items[2]
    regions = [Region(tag_id = tag_dict[tag].id, left=x,top=y,width=w,height=h)]
    with open(filepath, mode='rb') as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

In [38]:
# Send in batched of 64
i = 0
while True:
    try:
        imgs = tagged_images_with_regions[i*64:(i+1)*64]
    except:
        break
    upload_result = trainer.create_images_from_files(project.id, images=imgs)
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        print(i)
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)
    i += 1

Image batch upload failed.
88
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OKDuplicate
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image s

CustomVisionErrorException: No valid image files

In [41]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

In [47]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import os
# Now there is a trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

# Open the sample image and get back the prediction results.
for file in os.listdir("D:/val2014_subset"):    
    with open(f"D:/val2014_subset/{file}", mode="rb") as test_data:
        results = predictor.detect_image(project.id, publish_iteration_name, test_data)

    # Display the results.    
    for prediction in results.predictions:
        print(file + "\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))

COCO_val2014_000000000544.jpg	bowl: 1.47% bbox.left = 0.18, bbox.top = 0.69, bbox.width = 0.18, bbox.height = 0.27
COCO_val2014_000000000987.jpg	bowl: 1.01% bbox.left = 0.75, bbox.top = 0.34, bbox.width = 0.25, bbox.height = 0.66
COCO_val2014_000000001180.jpg	bowl: 1.05% bbox.left = 0.33, bbox.top = 0.19, bbox.width = 0.41, bbox.height = 0.48
COCO_val2014_000000001425.jpg	bowl: 2.06% bbox.left = 0.03, bbox.top = 0.35, bbox.width = 0.69, bbox.height = 0.42
COCO_val2014_000000001682.jpg	bowl: 5.82% bbox.left = 0.60, bbox.top = 0.14, bbox.width = 0.14, bbox.height = 0.19
COCO_val2014_000000001682.jpg	orange: 3.23% bbox.left = 0.60, bbox.top = 0.14, bbox.width = 0.14, bbox.height = 0.19
COCO_val2014_000000001803.jpg	bowl: 1.51% bbox.left = 0.28, bbox.top = 0.38, bbox.width = 0.25, bbox.height = 0.17
COCO_val2014_000000001803.jpg	orange: 1.02% bbox.left = 0.28, bbox.top = 0.38, bbox.width = 0.25, bbox.height = 0.17
COCO_val2014_000000002139.jpg	bowl: 2.45% bbox.left = 0.00, bbox.top = 0.11,

COCO_val2014_000000009590.jpg	bowl: 2.05% bbox.left = 0.05, bbox.top = 0.14, bbox.width = 0.35, bbox.height = 0.80
COCO_val2014_000000009590.jpg	orange: 1.13% bbox.left = 0.05, bbox.top = 0.14, bbox.width = 0.35, bbox.height = 0.80
COCO_val2014_000000010114.jpg	bowl: 1.19% bbox.left = 0.23, bbox.top = 0.07, bbox.width = 0.21, bbox.height = 0.14
COCO_val2014_000000010114.jpg	bowl: 1.69% bbox.left = 0.21, bbox.top = 0.64, bbox.width = 0.22, bbox.height = 0.34
COCO_val2014_000000010114.jpg	bowl: 1.86% bbox.left = 0.00, bbox.top = 0.42, bbox.width = 0.22, bbox.height = 0.58
COCO_val2014_000000010123.jpg	bowl: 1.18% bbox.left = 0.20, bbox.top = 0.79, bbox.width = 0.06, bbox.height = 0.09
COCO_val2014_000000010219.jpg	bowl: 1.46% bbox.left = 0.12, bbox.top = 0.03, bbox.width = 0.66, bbox.height = 0.86
COCO_val2014_000000010428.jpg	bowl: 1.22% bbox.left = 0.15, bbox.top = 0.77, bbox.width = 0.63, bbox.height = 0.21
COCO_val2014_000000011360.jpg	bowl: 1.84% bbox.left = 0.19, bbox.top = 0.00, b

COCO_val2014_000000019358.jpg	bowl: 3.84% bbox.left = 0.01, bbox.top = 0.03, bbox.width = 0.63, bbox.height = 0.86
COCO_val2014_000000019432.jpg	bowl: 1.17% bbox.left = 0.08, bbox.top = 0.07, bbox.width = 0.22, bbox.height = 0.81
COCO_val2014_000000019444.jpg	bowl: 1.44% bbox.left = 0.26, bbox.top = 0.14, bbox.width = 0.13, bbox.height = 0.03
COCO_val2014_000000019444.jpg	bowl: 1.48% bbox.left = 0.27, bbox.top = 0.15, bbox.width = 0.13, bbox.height = 0.03
COCO_val2014_000000019456.jpg	bowl: 2.27% bbox.left = 0.65, bbox.top = 0.16, bbox.width = 0.34, bbox.height = 0.78
COCO_val2014_000000019456.jpg	orange: 1.22% bbox.left = 0.65, bbox.top = 0.16, bbox.width = 0.34, bbox.height = 0.78
COCO_val2014_000000019589.jpg	bowl: 2.66% bbox.left = 0.43, bbox.top = 0.21, bbox.width = 0.57, bbox.height = 0.79
COCO_val2014_000000019723.jpg	bowl: 1.05% bbox.left = 0.48, bbox.top = 0.04, bbox.width = 0.52, bbox.height = 0.27
COCO_val2014_000000019916.jpg	bowl: 2.48% bbox.left = 0.79, bbox.top = 0.14, b

COCO_val2014_000000025751.jpg	bowl: 2.97% bbox.left = 0.33, bbox.top = 0.00, bbox.width = 0.66, bbox.height = 0.74
COCO_val2014_000000025860.jpg	bowl: 1.93% bbox.left = 0.17, bbox.top = 0.05, bbox.width = 0.25, bbox.height = 0.83
COCO_val2014_000000025860.jpg	bowl: 1.27% bbox.left = 0.36, bbox.top = 0.66, bbox.width = 0.61, bbox.height = 0.32
COCO_val2014_000000025955.jpg	bowl: 1.13% bbox.left = 0.25, bbox.top = 0.06, bbox.width = 0.36, bbox.height = 0.37
COCO_val2014_000000025984.jpg	bowl: 4.56% bbox.left = 0.46, bbox.top = 0.60, bbox.width = 0.34, bbox.height = 0.40
COCO_val2014_000000025984.jpg	orange: 1.87% bbox.left = 0.46, bbox.top = 0.60, bbox.width = 0.34, bbox.height = 0.40
COCO_val2014_000000025986.jpg	bowl: 1.86% bbox.left = 0.02, bbox.top = 0.25, bbox.width = 0.52, bbox.height = 0.32
COCO_val2014_000000026371.jpg	bowl: 1.62% bbox.left = 0.00, bbox.top = 0.52, bbox.width = 0.17, bbox.height = 0.47
COCO_val2014_000000026371.jpg	orange: 1.13% bbox.left = 0.00, bbox.top = 0.52,

COCO_val2014_000000032932.jpg	bowl: 2.06% bbox.left = 0.56, bbox.top = 0.33, bbox.width = 0.23, bbox.height = 0.67
COCO_val2014_000000032932.jpg	orange: 1.04% bbox.left = 0.56, bbox.top = 0.33, bbox.width = 0.23, bbox.height = 0.67
COCO_val2014_000000032964.jpg	bowl: 1.89% bbox.left = 0.43, bbox.top = 0.64, bbox.width = 0.50, bbox.height = 0.36
COCO_val2014_000000032964.jpg	bowl: 2.90% bbox.left = 0.00, bbox.top = 0.03, bbox.width = 0.64, bbox.height = 0.94
COCO_val2014_000000032964.jpg	bowl: 1.06% bbox.left = 0.36, bbox.top = 0.04, bbox.width = 0.63, bbox.height = 0.93
COCO_val2014_000000033652.jpg	bowl: 1.21% bbox.left = 0.00, bbox.top = 0.00, bbox.width = 0.49, bbox.height = 0.22
COCO_val2014_000000033652.jpg	bowl: 1.27% bbox.left = 0.13, bbox.top = 0.11, bbox.width = 0.64, bbox.height = 0.83
COCO_val2014_000000034445.jpg	bowl: 1.20% bbox.left = 0.13, bbox.top = 0.66, bbox.width = 0.21, bbox.height = 0.34
COCO_val2014_000000034582.jpg	bowl: 1.00% bbox.left = 0.00, bbox.top = 0.05, b

...

COCO_val2014_000000550702.jpg	bowl: 1.39% bbox.left = 0.06, bbox.top = 0.74, bbox.width = 0.35, bbox.height = 0.14
COCO_val2014_000000550702.jpg	bowl: 1.35% bbox.left = 0.11, bbox.top = 0.75, bbox.width = 0.27, bbox.height = 0.23
COCO_val2014_000000550702.jpg	bowl: 1.20% bbox.left = 0.14, bbox.top = 0.84, bbox.width = 0.20, bbox.height = 0.16
COCO_val2014_000000550864.jpg	bowl: 1.73% bbox.left = 0.75, bbox.top = 0.40, bbox.width = 0.24, bbox.height = 0.32
COCO_val2014_000000550864.jpg	orange: 1.07% bbox.left = 0.75, bbox.top = 0.40, bbox.width = 0.24, bbox.height = 0.32
COCO_val2014_000000551001.jpg	bowl: 1.00% bbox.left = 0.11, bbox.top = 0.07, bbox.width = 0.23, bbox.height = 0.63
COCO_val2014_000000551001.jpg	bowl: 1.61% bbox.left = 0.28, bbox.top = 0.06, bbox.width = 0.25, bbox.height = 0.66
COCO_val2014_000000551001.jpg	bowl: 1.53% bbox.left = 0.46, bbox.top = 0.06, bbox.width = 0.25, bbox.height = 0.65
COCO_val2014_000000551001.jpg	bowl: 1.19% bbox.left = 0.65, bbox.top = 0.12, b

COCO_val2014_000000558641.jpg	bowl: 1.11% bbox.left = 0.53, bbox.top = 0.39, bbox.width = 0.21, bbox.height = 0.29
COCO_val2014_000000558641.jpg	bowl: 4.27% bbox.left = 0.00, bbox.top = 0.23, bbox.width = 0.27, bbox.height = 0.64
COCO_val2014_000000558641.jpg	orange: 2.72% bbox.left = 0.00, bbox.top = 0.23, bbox.width = 0.27, bbox.height = 0.64
COCO_val2014_000000558671.jpg	bowl: 2.19% bbox.left = 0.10, bbox.top = 0.64, bbox.width = 0.20, bbox.height = 0.36
COCO_val2014_000000558671.jpg	bowl: 7.16% bbox.left = 0.82, bbox.top = 0.04, bbox.width = 0.18, bbox.height = 0.84
COCO_val2014_000000558671.jpg	orange: 1.14% bbox.left = 0.10, bbox.top = 0.64, bbox.width = 0.20, bbox.height = 0.36
COCO_val2014_000000558671.jpg	orange: 3.23% bbox.left = 0.82, bbox.top = 0.04, bbox.width = 0.18, bbox.height = 0.84
COCO_val2014_000000558854.jpg	bowl: 1.26% bbox.left = 0.71, bbox.top = 0.00, bbox.width = 0.29, bbox.height = 0.54
COCO_val2014_000000558854.jpg	bowl: 1.43% bbox.left = 0.01, bbox.top = 0.0

COCO_val2014_000000565296.jpg	bowl: 1.52% bbox.left = 0.83, bbox.top = 0.76, bbox.width = 0.16, bbox.height = 0.24
COCO_val2014_000000565395.jpg	bowl: 1.15% bbox.left = 0.00, bbox.top = 0.35, bbox.width = 0.25, bbox.height = 0.65
COCO_val2014_000000565395.jpg	bowl: 5.38% bbox.left = 0.30, bbox.top = 0.67, bbox.width = 0.41, bbox.height = 0.33
COCO_val2014_000000565395.jpg	orange: 1.65% bbox.left = 0.30, bbox.top = 0.67, bbox.width = 0.41, bbox.height = 0.33
COCO_val2014_000000565571.jpg	bowl: 1.25% bbox.left = 0.56, bbox.top = 0.19, bbox.width = 0.03, bbox.height = 0.14
COCO_val2014_000000565903.jpg	bowl: 1.68% bbox.left = 0.47, bbox.top = 0.19, bbox.width = 0.15, bbox.height = 0.05
COCO_val2014_000000565903.jpg	orange: 1.10% bbox.left = 0.47, bbox.top = 0.19, bbox.width = 0.15, bbox.height = 0.05
COCO_val2014_000000566282.jpg	bowl: 1.06% bbox.left = 0.15, bbox.top = 0.77, bbox.width = 0.08, bbox.height = 0.10
COCO_val2014_000000566282.jpg	bowl: 1.06% bbox.left = 0.53, bbox.top = 0.01,

COCO_val2014_000000570465.jpg	bowl: 2.26% bbox.left = 0.80, bbox.top = 0.55, bbox.width = 0.16, bbox.height = 0.39
COCO_val2014_000000570465.jpg	bowl: 1.05% bbox.left = 0.01, bbox.top = 0.14, bbox.width = 0.66, bbox.height = 0.81
COCO_val2014_000000570465.jpg	orange: 1.15% bbox.left = 0.80, bbox.top = 0.55, bbox.width = 0.16, bbox.height = 0.39
COCO_val2014_000000570848.jpg	bowl: 1.23% bbox.left = 0.50, bbox.top = 0.68, bbox.width = 0.16, bbox.height = 0.20
COCO_val2014_000000570848.jpg	bowl: 1.85% bbox.left = 0.48, bbox.top = 0.29, bbox.width = 0.31, bbox.height = 0.69
COCO_val2014_000000571219.jpg	bowl: 2.90% bbox.left = 0.00, bbox.top = 0.26, bbox.width = 0.34, bbox.height = 0.58
COCO_val2014_000000571219.jpg	orange: 1.56% bbox.left = 0.00, bbox.top = 0.26, bbox.width = 0.34, bbox.height = 0.58
COCO_val2014_000000571449.jpg	bowl: 1.41% bbox.left = 0.41, bbox.top = 0.09, bbox.width = 0.43, bbox.height = 0.37
COCO_val2014_000000571585.jpg	bowl: 2.11% bbox.left = 0.08, bbox.top = 0.08,

COCO_val2014_000000577377.jpg	bowl: 1.76% bbox.left = 0.22, bbox.top = 0.55, bbox.width = 0.10, bbox.height = 0.09
COCO_val2014_000000577451.jpg	bowl: 1.21% bbox.left = 0.00, bbox.top = 0.64, bbox.width = 0.69, bbox.height = 0.35
COCO_val2014_000000577539.jpg	bowl: 1.35% bbox.left = 0.22, bbox.top = 0.02, bbox.width = 0.61, bbox.height = 0.87
COCO_val2014_000000577654.jpg	bowl: 1.86% bbox.left = 0.63, bbox.top = 0.06, bbox.width = 0.27, bbox.height = 0.80
COCO_val2014_000000577847.jpg	bowl: 1.31% bbox.left = 0.55, bbox.top = 0.08, bbox.width = 0.33, bbox.height = 0.77
COCO_val2014_000000577853.jpg	bowl: 1.14% bbox.left = 0.05, bbox.top = 0.00, bbox.width = 0.35, bbox.height = 0.47
COCO_val2014_000000577853.jpg	bowl: 1.38% bbox.left = 0.15, bbox.top = 0.09, bbox.width = 0.76, bbox.height = 0.70
COCO_val2014_000000577864.jpg	bowl: 1.64% bbox.left = 0.82, bbox.top = 0.21, bbox.width = 0.04, bbox.height = 0.05
COCO_val2014_000000577864.jpg	orange: 1.08% bbox.left = 0.82, bbox.top = 0.21, b